In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
file_path = './Final_data.csv'
data = pd.read_csv(file_path,encoding = 'utf-8-sig')

In [3]:
data.columns

Index(['Type', 'Sentence', 'Quality', 'Price', 'Packaging', 'Service',
       'Sentiment'],
      dtype='object')

In [4]:
df = []
for i, row in data.iterrows():
  check = True
  for index in range(4):
    if row.iloc[index + 2] != 0:
      df.append([row['Sentence'] , row['Sentiment'], index + 1])
      check = False
  if check:
    df.append([row['Sentence'] , row['Sentiment'], 0])

In [5]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Remove Word have NOT
blabla = ['weren',"weren't",'doesn','hasn',"hasn't",'haven',"haven't","isn't",
          "won't",'wouldn',"wouldn't",
          "aren't", "couldn't",'couldn', 'didn', "didn't",'doesn',"doesn't",'don',"don't",'hadn',"hadn't",
          'isn',"isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',
          "shouldn't",'wasn',"wasn't", "didn't",'not', 'nor','aren']
for word in blabla:
  if(word in stop_words):
    stop_words.remove(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
from nltk.stem import LancasterStemmer
wnl = WordNetLemmatizer()
stemmer = LancasterStemmer()

# Word #

word_dict = {"n't" : " not","'re" : " are","'s" : " is", "'ve" : " have"}
def replaceWord(Word):
  w = ''
  run = False
  for word in Word:
    if word == "'":
      run = True
  if run:
    for word in word_dict:
      Word = Word.replace(word,word_dict[word])
  return Word

# Sentences #

def replaceDot(Sen):
  w = ''
  for word in Sen:
    if (ord(word) in range(33,48)) or (ord(word) in range(58,65)) or (ord(word) in range(91,97)):
      if word == "'":
        w += word
      else:
        w += '.'
    else:
      w += word.lower()
  return replaceWord(w)

Links = ['and','but','howerver']
def splitSen(Sen):
  temp = Sen.split('.')
  temp_2 = []
  for sen in temp:
    s = ''
    for word in sen.split():
      if word in Links:
        temp_2.append(s[:-1])
        s = ''
      else:
        s += word + ' '
    if s != '':
      temp_2.append(s[:-1])

  return temp_2

def removeStopWord(Sen):
  s = ''
  for word in Sen.split():
    if word not in stop_words:
      s += stemmer.stem(word) + ' '

  return s[:-1]

# Main Pre-processing Sen #

def ProcessReview(Review):
  Review = replaceDot(Review)
  temp = splitSen(Review)
  for index in range(len(temp)):
    temp[index] = removeStopWord(temp[index])

  return temp

In [7]:
for index in range(len(df)):
  try:
    df[index][0] =  removeStopWord(df[index][0])
  except:
    print(index)

In [8]:
cols = ['Key','Amount','Negative','Neutral','Positive','None','Quality','Price','Packaging','Service']
word_code = {}
code = 0
Keys = []

for index in range(len(df)):
  for word in df[index][0].split():
    lb = [1.,0.,0.,0.,0.,0.,0.,0.,0.]
    lb[df[index][1] + 1] += 1
    lb[df[index][2] + 4] += 1
    if word not in word_code:
      word_code[word] = code
      code += 1
      Keys.append([word] + lb)
    else:
      for i in range(8):
        Keys[word_code[word]][i + 1] += lb[i]


In [9]:
Keys = np.array(Keys)
Keys = pd.DataFrame(Keys,columns = cols)
Keys.iloc[:,1:] = Keys.iloc[:,1:].astype(float)

In [10]:
Keys.describe()

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
count,2141,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0
unique,2141,53.0,27.0,21.0,33.0,17.0,31.0,18.0,16.0,2.0
top,rec,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
freq,1,1255.0,1121.0,1207.0,1076.0,1406.0,1045.0,1625.0,1664.0,1799.0


In [11]:
Keys_2 = Keys.copy()
for index in range(Keys.shape[0]):
  temp = Keys_2.iloc[index,2:5].sum()
  if temp != 0:
    Keys_2.iloc[index,2:5] /= temp
  temp = Keys_2.iloc[index,-5:].sum()
  if temp != 0:
    Keys_2.iloc[index,-5:] /= temp

  a = Keys_2.loc[index,'Negative']
  b = Keys_2.loc[index,'Neutral']
  c = Keys_2.loc[index,'Positive']
  temp = abs(a - b) + abs(a - c) + abs(b - c)
  if temp == 0:
    Keys_2.iloc[index,2:5] /= 0.001
  else:
    Keys_2.iloc[index,2:5] /= temp

Keys_2.describe()

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
count,2141,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0
unique,2141,53.0,101.0,98.0,95.0,83.0,90.0,76.0,77.0,22.0
top,rec,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
freq,1,1255.0,1121.0,1207.0,1076.0,1406.0,1045.0,1625.0,1664.0,1799.0


In [12]:
Keys_2.loc[Keys_2['Key'] == stemmer.stem('great')]

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
116,gre,59.0,0.009259,0.027778,0.509259,0.061224,0.285714,0.530612,0.122449,0.0


In [13]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0, 5))

# Keys_2[cols[2:]]= scaler.fit_transform(Keys_2[cols[2:]])
# Keys_2.describe()

In [14]:
def sen2vec(Sen,ver = 0):
  temp = pd.DataFrame(columns = cols)
  for word in Sen.split():
    if Keys_2.loc[Keys_2['Key'] == word].empty == False:
      temp = pd.concat([temp,Keys_2.loc[Keys_2['Key'] == word]], axis = 0)

  if temp.empty:
    vec = [0,0,0,0,0,0,0,0]
    return np.array(vec).reshape(1,8)

  result = temp.iloc[:,2:].sum().values.reshape(1,8)
  if ver != 0:
    result[2:] = result[2:] / temp.shape[0]
  return result[0]

# Main #

X = sen2vec(df[0][0])
y = np.array([df[0][1],df[0][2]]).reshape(1,2)
for index in range(len(df)):
  X = np.vstack((X,sen2vec(df[index][0],1)))
  y = np.vstack((y,np.array([df[index][1],df[index][2]]).reshape(1,2)))

In [15]:
# X_sentiment = X[:,:3]
# X_aspect = X[:,3:]
# y_sentiment = y[:,0]
# y_aspect = y[:,1]

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [17]:
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn import svm
# from sklearn.neighbors import KNeighborsClassifier

# # svm
# param_grid = {'kernel': ['linear', 'rbf'], 'C': [1, 10, 100], 'gamma': [0.1, 0.01, 0.001]}
# # knn
# # param_grid = {'n_neighbors': [1,3, 5, 7,9,11,13,15], 'weights': ['uniform', 'distance']}

# model = svm.SVC()

# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_search.fit(X_train[:,3:], y_train[:,1])

# results = pd.DataFrame(grid_search.cv_results_)


# # Print the results svm
# print(results[['param_kernel','param_C', 'param_gamma', 'mean_test_score']])
# # Print the results knn
# # print(results[['param_n_neighbors', 'param_weights', 'mean_test_score']])

In [18]:
# from sklearn import svm

# model_aspect = svm.SVC(kernel = 'linear',gamma = 0.1, C = 100)
# model_aspect.fit(X_train[:,3:],y_train[:,1])

# model_sentiment = []
# for aspect in range(5):
#   temp = []
#   for index in range(len(y_train)):
#     if y_train[index,1] == aspect:
#       temp.append(index)
#   x_temp = X_train[temp,:3]
#   y_temp = y_train[temp,0]
#   model = svm.SVC(kernel = 'linear',gamma = 0.1, C = 100)
#   model.fit(x_temp,y_temp)
#   model_sentiment.append(model)

In [39]:
# from sklearn.neighbors import KNeighborsClassifier

# model_aspect = KNeighborsClassifier(n_neighbors = 3)
# model_aspect.fit(X_train[:,3:],y_train[:,1])

# model_sentiment = []
# for aspect in range(5):
#   temp = []
#   for index in range(len(y_train)):
#     if y_train[index,1] == aspect:
#       temp.append(index)
#   x_temp = X_train[temp,:3]
#   y_temp = y_train[temp,0]
#   model = KNeighborsClassifier(n_neighbors = 3)
#   model.fit(x_temp,y_temp)
#   model_sentiment.append(model)

In [49]:
from sklearn.linear_model import LogisticRegression

model_aspect = LogisticRegression()
model_aspect.fit(X_train[:,3:],y_train[:,1])

model_sentiment = []
for aspect in range(5):
  temp = []
  for index in range(len(y_train)):
    if y_train[index,1] == aspect:
      temp.append(index)
  x_temp = X_train[temp,:3]
  y_temp = y_train[temp,0]
  model = LogisticRegression()
  model.fit(x_temp,y_temp)
  model_sentiment.append(model)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [50]:
def predict(Vec):
  aspect = model_aspect.predict(Vec[3:].reshape(1,5))
  sentiment = model_sentiment[aspect[0]].predict(Vec[:3].reshape(1,3))

  return [sentiment[0],aspect[0]]

y_pred = []
for item in X_test:
  y_pred.append(predict(item.reshape(1,8)[0]))

y_pred = np.array(y_pred).reshape(-1,2)

In [51]:
def cal_accuracy(matrix,ver = 0):
  sum = matrix[0][0] + matrix[0][1] + matrix[1][0] + matrix[1][1]
  sum_pos = matrix[1][0] + matrix[1][1]

  accuracy = (matrix[0][0] + matrix[1][1]) / sum
  precision = matrix[1][1] / (matrix[1][1] + matrix[0][1])
  recall = matrix[1][1] / (matrix[1][1] + matrix[1][0])
  f1_score = (2 * precision * recall) / (precision + recall)

  if ver == 0:
    return np.array([accuracy,precision,recall,f1_score]).reshape(1,4) * sum_pos / sum
  return np.array([accuracy,precision,recall,f1_score]).reshape(1,4)

# sentiment accuracty #
result = np.array([0.,0.,0.,0.]).reshape(1,4)
for item in range(3):
  matrix = [[0,0],[0,0]]
  for index in range(len(y_test)):
    a = 1 if y_test[index,0] == item else 0
    b = 1 if y_pred[index,0] == item else 0
    matrix[a][b] += 1
  result = result + cal_accuracy(matrix)
print('sentiment',result[0])

for item in range(5):
  matrix = [[0,0],[0,0]]
  for index in range(len(y_test)):
    a = 1 if y_test[index,1] == item else 0
    b = 1 if y_pred[index,1] == item else 0
    matrix[a][b] += 1
  print(matrix)
  print(item,cal_accuracy(matrix,1)[0])

sentiment [0.93940446 0.91085053 0.91056911 0.91049929]
[[183, 10], [7, 46]]
0 [0.93089431 0.82142857 0.86792453 0.8440367 ]
[[154, 10], [8, 74]]
1 [0.92682927 0.88095238 0.90243902 0.89156627]
[[208, 2], [3, 33]]
2 [0.9796748  0.94285714 0.91666667 0.92957746]
[[211, 8], [3, 24]]
3 [0.95528455 0.75       0.88888889 0.81355932]
[[193, 5], [14, 34]]
4 [0.92276423 0.87179487 0.70833333 0.7816092 ]


In [52]:
from sklearn.metrics import classification_report

print('sentiment')
print(classification_report(y_test[:,0],y_pred[:,0], digits=4))
print('aspect')
print(classification_report(y_test[:,1],y_pred[:,1], digits=4))

sentiment
              precision    recall  f1-score   support

           0     0.9136    0.9136    0.9136        81
           1     0.9231    0.8824    0.9023        68
           2     0.9000    0.9278    0.9137        97

    accuracy                         0.9106       246
   macro avg     0.9122    0.9079    0.9098       246
weighted avg     0.9109    0.9106    0.9105       246

aspect
              precision    recall  f1-score   support

           0     0.8214    0.8679    0.8440        53
           1     0.8810    0.9024    0.8916        82
           2     0.9429    0.9167    0.9296        36
           3     0.7500    0.8889    0.8136        27
           4     0.8718    0.7083    0.7816        48

    accuracy                         0.8577       246
   macro avg     0.8534    0.8569    0.8521       246
weighted avg     0.8610    0.8577    0.8569       246



In [53]:
# 0     0.8197    0.9434    0.8772        53
#            1     0.8824    0.9146    0.8982        82
#            2     0.9143    0.8889    0.9014        36
#            3     0.7419    0.8519    0.7931        27
#            4     0.9412    0.6667    0.7805        48

In [54]:
count = 0
for index in range(len(y_test)):
  if y_test[index][0] == y_pred[index][0] and y_test[index][1] == y_pred[index][1]:
    count += 1
count / len(y_test)

0.7886178861788617

In [55]:
Keys.loc[Keys['Key'] == 'not']

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
145,not,127.0,80.0,30.0,17.0,44.0,43.0,12.0,9.0,0.0


In [56]:
Keys_2.loc[Keys_2['Key'] == 'not']

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
145,not,127.0,0.634921,0.238095,0.134921,0.407407,0.398148,0.111111,0.083333,0.0


In [58]:
def analyReview(Review):
  sens = ProcessReview(Review)
  sens_v2 = []
  for sen in sens:
    if sen.isspace() == False and sen != '':
      sens_v2.append(sen)

  lb = []
  for sen in sens_v2:
    lb.append(predict(sen2vec(sen)))

  for index in range(len(lb)):
    print(sens_v2[index],lb[index])

text = "The taste was bad, but the taste is not bad. The expiration date is 2023. It’s flat."
# text = "Nothing fancy here, just exactly what you'd expect - 16.9oz bottles of Coca Cola. Saved me a trip to the store. Definitely worth."

analyReview(text)

tast bad [0, 1]
tast not bad [0, 1]
expir dat 2023 [2, 3]
it’s flat [0, 1]
